In [ ]:
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from ta import add_all_ta_features
from sklearn.metrics import mean_squared_error
from math import sqrt

# Initialize Alpha Vantage TimeSeries
api_key = 'YOUR_ALPHA_VANTAGE_API_KEY'
ts = TimeSeries(key=api_key, output_format='pandas')

# Get historical stock data
stock_symbol = 'AAPL'
data, meta_data = ts.get_daily(symbol=stock_symbol, outputsize='full')
data = data['4. close']  # Close prices

# Feature Engineering with Technical Indicators
data = add_all_ta_features(data, open="1. open", high="2. high", low="3. low", close="4. close", volume="5. volume")

# Handle any infinite or NaN values
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)

# Target Variable (Next day's price)
data['target'] = data['4. close'].shift(-1)

# Splitting data into train and test sets
train = data[:int(0.8*len(data))]
test = data[int(0.8*len(data)):]

X_train = train.drop('target', axis=1)
y_train = train['target']
X_test = test.drop('target', axis=1)
y_test = test['target']

# Model: Random Forest Regressor
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
rmse = sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Square Error: {rmse}")

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(test.index, y_test, label='Actual Price')
plt.plot(test.index, y_pred, label='Predicted Price', color='r')
plt.title(f'{stock_symbol} Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

# Backtesting (Simple Example)
# Buy if the predicted price for the next day is higher than the current closing price
cash = 10000  # Starting cash
shares = 0
portfolio_value = []

for i in range(len(test) - 1):
    if y_pred[i] > test.iloc[i]['4. close'] and cash >= test.iloc[i]['4. close']:
        shares_bought = cash // test.iloc[i]['4. close']
        shares += shares_bought
        cash -= shares_bought * test.iloc[i]['4. close']
    elif y_pred[i] < test.iloc[i]['4. close'] and shares > 0:
        cash += shares * test.iloc[i]['4. close']
        shares = 0
    portfolio_value.append(cash + shares * test.iloc[i]['4. close'])

plt.figure(figsize=(12, 6))
plt.plot(test.index[:-1], portfolio_value, label='Portfolio Value')
plt.title('Backtest Result')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.show()
